# Bucketing
- Dividing the data into manageable chunks for better performance, similar to partitioning
- Main formula for bucketing
  - $hash(bucketingColValue)mod(no. of buckets)$
  - hash function gives a large value and with the mod function it can be brought within the limits

## How Bucketing Helps in Joins
- If you store your data without partitioning or bucketing, join operations will typically involve **shuffle, sort, and merge** steps, which are computationally expensive due to the need to redistribute and sort data across nodes.

- If you partition your data on a high cardinality column, it can lead to the **small file problem**, where each partition ends up being very small and inefficient to process

- To avoid the limitations of partitioning, you can bucket your data by specifying a fixed number of buckets. Bucketing works well for join optimization, especially for large datasets.
    - Bucketing works by applying a hash function to the column used for bucketing (e.g., `product_id`). The hash function ensures that the same value (e.g., `product_id = 1`) always maps to the same bucket. As a result, rows with the same `product_id` from two tables (e.g., `Orders` and `Products`) will be stored in the same bucket. This eliminates the need for shuffling during a join operation, significantly improving performance.

### Optimal Conditions for Joins on Bucketed Tables
1. If both tables are bucketed on the same key and have the same number of buckets => **No shuffle**
2. If both tables are bucketed on the same key but have different number of buckets => **One of the tables would be suffled**
3. If both tables are bucketed but on different keys even though they have same number of buckets => **Full Shuffle**



## How to Decide the Number of Buckets?

- The **optimal bucket size** is typically between **128 MB and 200 MB**, depending on your system's configuration and the underlying file system (e.g., HDFS).

- To calculate the **number of buckets**, use the formula:  
  \[
  \text{Number of Buckets} = \frac{\text{Size of Dataset (in MB)}}{\text{Optimal Bucket Size (in Mnteg#er.

---

### Example:
If your dataset is 10 GB (10,240 MB) and the optimal bucket size is 200 MB:

\[
\text{Number of Buckets} = \frac{10,240}{200} = 51.2
\]

You would set the number of buckets to **51**. 

This ensures that each bucket is sized appropriately for efficient processing while avoiding overhead caused by excessively small or large buckets.sively small or large buckets.

In [1]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .config("spark.driver.memory","10g")
         .master("local[*]")
         .appName("bucketing")
         .getOrCreate()
        )

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [3]:
orders_file = "orders.csv"
df_orders = spark.read.csv(orders_file, header=True, inferSchema=True)
df_orders.show(5)

+--------+----------+-----------+--------+-------------------+------------+
|order_id|product_id|customer_id|quantity|         order_date|total_amount|
+--------+----------+-----------+--------+-------------------+------------+
|       1|        80|         10|       4|2023-03-20 00:00:00|        1003|
|       2|        69|         30|       3|2023-12-11 00:00:00|         780|
|       3|        61|         20|       4|2023-04-26 00:00:00|        1218|
|       4|        62|         44|       3|2023-08-26 00:00:00|        2022|
|       5|        78|         46|       4|2023-08-05 00:00:00|        1291|
+--------+----------+-----------+--------+-------------------+------------+
only showing top 5 rows



In [4]:
products_file = "products.csv"
df_products = spark.read.csv(products_file, header=True, inferSchema=True)
df_products.show(5)

+----------+------------+-----------+-------+-----+-----+
|product_id|product_name|   category|  brand|price|stock|
+----------+------------+-----------+-------+-----+-----+
|         1|   Product_1|Electronics|Brand_4|   26|  505|
|         2|   Product_2|    Apparel|Brand_4|  489|   15|
|         3|   Product_3|    Apparel|Brand_4|  102|  370|
|         4|   Product_4|  Groceries|Brand_1|   47|  433|
|         5|   Product_5|  Groceries|Brand_3|  244|  902|
+----------+------------+-----------+-------+-----+-----+
only showing top 5 rows



## Comparing Join Operation with and w/o Bucketing

### W/O Bucketing

In [17]:
# set broadcast off for now
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [9]:
df_orders.join(df_products, on="product_id", how="inner").explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (11)
+- Project (10)
   +- SortMergeJoin Inner (9)
      :- Sort (4)
      :  +- Exchange (3)
      :     +- Filter (2)
      :        +- Scan csv  (1)
      +- Sort (8)
         +- Exchange (7)
            +- Filter (6)
               +- Scan csv  (5)


(1) Scan csv 
Output [6]: [order_id#17, product_id#18, customer_id#19, quantity#20, order_date#21, total_amount#22]
Batched: false
Location: InMemoryFileIndex [file:/home/jovyan/orders.csv]
PushedFilters: [IsNotNull(product_id)]
ReadSchema: struct<order_id:int,product_id:int,customer_id:int,quantity:int,order_date:timestamp,total_amount:int>

(2) Filter
Input [6]: [order_id#17, product_id#18, customer_id#19, quantity#20, order_date#21, total_amount#22]
Condition : isnotnull(product_id#18)

(3) Exchange
Input [6]: [order_id#17, product_id#18, customer_id#19, quantity#20, order_date#21, total_amount#22]
Arguments: hashpartitioning(product_id#18, 200), ENSURE_REQUIREMENTS, [plan_id=118]

(4) Sort
Inpu

***Steps 3 and 7 show the two full shuffles***

### With Bucketing

In [10]:
df_orders.write.bucketBy(4, col="product_id").mode("overwrite").saveAsTable("orders_buckected")

In [11]:
df_products.write.bucketBy(4, col="product_id").mode("overwrite").saveAsTable("products_buckected")

In [12]:
df_orders_bucketed = spark.table("orders_buckected")
df_products_bucketed = spark.table("products_buckected")

In [13]:
df_orders_bucketed.join(df_products_bucketed,
                        on="product_id", 
                        how="inner").explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (9)
+- Project (8)
   +- SortMergeJoin Inner (7)
      :- Sort (3)
      :  +- Filter (2)
      :     +- Scan parquet spark_catalog.default.orders_buckected (1)
      +- Sort (6)
         +- Filter (5)
            +- Scan parquet spark_catalog.default.products_buckected (4)


(1) Scan parquet spark_catalog.default.orders_buckected
Output [6]: [order_id#209, product_id#210, customer_id#211, quantity#212, order_date#213, total_amount#214]
Batched: true
Bucketed: true
Location: InMemoryFileIndex [file:/home/jovyan/spark-warehouse/orders_buckected]
PushedFilters: [IsNotNull(product_id)]
ReadSchema: struct<order_id:int,product_id:int,customer_id:int,quantity:int,order_date:timestamp,total_amount:int>
SelectedBucketsCount: 4 out of 4

(2) Filter
Input [6]: [order_id#209, product_id#210, customer_id#211, quantity#212, order_date#213, total_amount#214]
Condition : isnotnull(product_id#210)

(3) Sort
Input [6]: [order_id#209, product_id#210, customer_id#211

***Voila! No Exchange?!!***

### Summary: Comparing Join Operations With and Without Bucketing

When comparing join operations in Spark with and without bucketing, the key differences lie in the handling of data shuffling and the efficiency of the execution plan.

#### **Without Bucketing**
- The join operation involves **shuffle-sort-merge**, which is resource-intensive due to the following:
  - **Exchange**: Data is shuffled across the cluster based on the join key (`product_id`).
  - **Sort**: Both datasets are sorted after the shuffle to enable the merge.
  - The physical plan shows multiple stages, including **Exchange** and **Sort** steps, making the process more expensive in terms of time and computation.

#### **With Bucketing**
- When data is bucketed on the join key (`product_id`):
  - Both datasets are pre-partitioned into the same number of buckets, ensuring that matching keys are stored in the same bucket.
  - The **Exchange** step is eliminated as there is no need for data shuffling during the join.
  - The physical plan is simplified, showing only the **SortMergeJoin**, with the absence of **Exchange**, resulting in a more efficient execution.


## Comparing Aggregation Operation with and w/o Bucketing

### W/O Bucketing

In [15]:
df_orders_grouped = df_orders.groupby("product_id").agg(F.sum("total_amount").alias("total_amount"))
df_orders_grouped.show(5)
df_orders_grouped.explain(mode='formatted')

+----------+------------+
|product_id|total_amount|
+----------+------------+
|        31|        9994|
|        85|       10630|
|        65|       12806|
|        53|       14410|
|        78|       12652|
+----------+------------+
only showing top 5 rows

== Physical Plan ==
AdaptiveSparkPlan (5)
+- HashAggregate (4)
   +- Exchange (3)
      +- HashAggregate (2)
         +- Scan csv  (1)


(1) Scan csv 
Output [2]: [product_id#18, total_amount#22]
Batched: false
Location: InMemoryFileIndex [file:/home/jovyan/orders.csv]
ReadSchema: struct<product_id:int,total_amount:int>

(2) HashAggregate
Input [2]: [product_id#18, total_amount#22]
Keys [1]: [product_id#18]
Functions [1]: [partial_sum(total_amount#22)]
Aggregate Attributes [1]: [sum#287L]
Results [2]: [product_id#18, sum#288L]

(3) Exchange
Input [2]: [product_id#18, sum#288L]
Arguments: hashpartitioning(product_id#18, 200), ENSURE_REQUIREMENTS, [plan_id=317]

(4) HashAggregate
Input [2]: [product_id#18, sum#288L]
Keys [1]: [produc

***There is an exchange involved***

### With Bucketing

In [16]:
df_orders_bucketed_grouped = df_orders_bucketed.groupby("product_id").agg(F.sum("total_amount").alias("total_amount"))
df_orders_bucketed_grouped.show(5)
df_orders_bucketed_grouped.explain(mode='formatted')

+----------+------------+
|product_id|total_amount|
+----------+------------+
|        46|       10902|
|        65|       12806|
|        38|       14283|
|        95|       20949|
|        12|       17187|
+----------+------------+
only showing top 5 rows

== Physical Plan ==
AdaptiveSparkPlan (4)
+- HashAggregate (3)
   +- HashAggregate (2)
      +- Scan parquet spark_catalog.default.orders_buckected (1)


(1) Scan parquet spark_catalog.default.orders_buckected
Output [2]: [product_id#210, total_amount#214]
Batched: true
Bucketed: true
Location: InMemoryFileIndex [file:/home/jovyan/spark-warehouse/orders_buckected]
ReadSchema: struct<product_id:int,total_amount:int>
SelectedBucketsCount: 4 out of 4

(2) HashAggregate
Input [2]: [product_id#210, total_amount#214]
Keys [1]: [product_id#210]
Functions [1]: [partial_sum(total_amount#214)]
Aggregate Attributes [1]: [sum#312L]
Results [2]: [product_id#210, sum#313L]

(3) HashAggregate
Input [2]: [product_id#210, sum#313L]
Keys [1]: [produ

***No exchange***

### Comparison of GroupBy Operation with and without Bucketing

#### **Without Bucketing**
- **Physical Plan Overview:**
  - **Exchange Stage (Step 3):** A `hashpartitioning` operation is required to distribute the data based on the `product_id` column across partitions. This ensures that all rows with the same `product_id` are grouped together for aggregation.
  - **Multiple Aggregation Stages:**
    - **Partial Aggregation (Step 2):** A `HashAggregate` operation is performed at the initial stage to calculate partial sums for each partition.
    - **Final Aggregation (Step 4):** Another `HashAggregate` combines the partial results after data has been shuffled and grouped in the exchange stage.
  - **Data Source (Step 1):** A CSV file is scanned, and the schema indicates the data includes `product_id` and `total_amount`.

- **Inefficiencies:**
  - **Exchange Overhead:** The `Exchange` stage involves shuffling data across the cluster, which can be expensive in terms of time and resources.
  - **Non-Bucketed Source:** The data lacks inherent structure for optimized partitioning, requiring additional computational effort.

---

#### **With Bucketing**
- **Physical Plan Overview:**
  - **No Exchange Stage:** The exchange operation is eliminated due to pre-bucketing of the data on the `product_id` column. This ensures data is already distributed into buckets during the write stage.
  - **Efficient Aggregation Stages:**
    - **Partial Aggregation (Step 2):** Similar to the non-bucketed case, partial sums are calculated for each bucket.
    - **Final Aggregation (Step 3):** Combines results from partial aggregations directly without requiring shuffling.
  - **Data Source (Step 1):** Data is read from a pre-bucketed Parquet file, with `Batched` and `Bucketed` optimizations applied. The `SelectedBucketsCount` confirms that all buckets are used.

- **Efficiencies:**
  - **Eliminated Exchange Stage:** Data is already partitioned into buckets, which avoids the costly shuffling process.
  - **Optimized Reads:** The bucketed data structure enables faster access and processing.

---

### **Key Observations**
| **Aspect**           | **Without Bucketing**                                          | **With Bucketing**                                       |
|-----------------------|---------------------------------------------------------------|---------------------------------------------------------|
| **Exchange Operation**| Present, requires `hashpartitioning`.                         | Absent, as data is already bucketed.                   |
| **Data Shuffling**    | Significant overhead due to repartitioning.                   | Eliminated due to pre-bucketing.                       |
| **Aggregation**       | Two stages: Partial + Final aggregation.                      | Two stages, but no shuffling needed between them.       |
| **Storage Format**    | CSV (non-bucketed), slower reads.                             | Parquet (bucketed), supports batched and optimized reads. |
| **Performance Impact**| Higher computational and I/O overhead.                       | Faster and more efficient due to bucketed partitioning. |

---

### **Conclusion**
Bucketing significantly improves the efficiency of `groupBy` operations by eliminating the need for data shuffling (`Exchange` stage) and leveraging optimized storage formats like Parquet. This optimization is particularly beneficial for large datasets where reducing shuffle and partitioning overhead can lead to substantial performance gains.